In [ ]:
import pennylane as qml
import pennylane.numpy as np

First approach

In [2]:
def half_life(gamma, p):
    dt = 0.0001
    n = 0
    f = 0

    while f < 1/(2*dt*gamma*(2*p-1)):
        f += (1-dt*gamma)**n
        n+=1
    return dt*n


# These are the public test cases
test_cases = [
    ('[0.1,0.92]', '9.05'),
    ('[0.2,0.83]', '7.09')
]

In [1]:
gamma, p = [0.1, 0.92]

In [ ]:
half_life(gamma,p)

Second approach

In [ ]:
def half_life(gamma, p):
    num_wires = 1

    dev = qml.device("default.mixed", wires=num_wires)

    @qml.qnode(dev)
    def noise(gamma,p,dt,n):
        qml.Hadamard(wires=0)
        for _ in range(n):
            qml.GeneralizedAmplitudeDamping(gamma* dt,p,wires=0 )
        return qml.probs(wires=0)
    dt = 0.001
    tmin =0
    tmax = 100
    while(tmax-tmin)> dt:
        tmid = (tmax+tmin)/2
        prob = noise(gamma, p, dt, int(tmid/dt))[1]
        if prob > 1/4:
            tmin = tmid
        else:
            tmax=tmid

    return tmid

In [ ]:
half_life(gamma, p)

## Don't Hit the Ground

**Backstory**

Zenda and Reece discuss strategies to interfere with the correct functioning of Sqynet, the conscious quantum computer that's taking over the galaxy. One way to tamper with its hardware is to bombard Sqynet's outer shell with plasma grenades, exposing the quantum computer to higher temperatures. As a consequence, Sqynet won't be able to prepare its ground state quickly.

**Preparing ground states**

Preparing a fiducial state, usually denoted by $\Ket{0}$ is the first step before carrying out any quantum computations. For most quantum computers, this is a straightforward process (although sometimes energy and time consuming). We need to bring the quantum device to almost absolute zero so that it relaxes to its ground state —the state of minimal energy— which is our choice of fiducial state.

Why does this happen? Quantum systems are never really isolated, so they will exchange energy with their environment. The net effect is that any quantum properties of the system's state, i.e. superpositions and entanglement, are lost after some time.

How do we model this energy exchange at finite temperature? The [Generalized Amplitude Damping](https://docs.pennylane.ai/en/stable/code/api/pennylane.GeneralizedAmplitudeDamping.html) channel provides a good approximation. Suppose $\gamma$ is the photon loss rate at zero temperature, and $p$ is the probability that a qubit emits a photon to the finite-temperature environment (the system will also absorb photons with probability $1-p$). We can approximate the interaction with the environment for a duration $t$ via the circuit below.
![circuit](./images/Don't%20Hit%20the%20Ground_1.png\)

That is, we compose many Generalized Amplitude Damping channels with infinitesimal noise parameters $\gamma \Delta t$ and de-excitation probability $p$. A shorter step $\Delta t$ gives a more precise calculation, but we will need more Generalized Amplitude Damping channels to model the same duration $T$.

Zenda and Reece need to figure out a measure of how quickly Sqynet can relax its fiducial state, given some photon loss rate $\gamma$ and emission probability $p$. Assuming that Sqynet is in the initial state
$$\Ket{+} = \frac{1}{\sqrt{2}}\Ket{0} + \frac{1}{\sqrt{2}\Ket{1}}$$
your task is to estimate the relaxation half-life, which is the time at which we obtain the outcome $\Ket{1}$ with probability 1/4 (the measurement is performed in the computational basis).

**Challenge code**
You must complete the half_life function to calculate the time $T$ at which the probability of measuring $\Ket{1}$ becomes 1/4.

**Input**
As input to this problem, you are given:

* gamma (float): The zero-temperature photon loss rate.
* p (float): The de-excitation probability due to temperature effects

**Output**
This code will output a float equal to your estimate of the relaxation half-life. Note that you may require the step and iterations of your circuit to actually reach the half-life.

In [3]:
import json
import pennylane as qml
import pennylane.numpy as np
def half_life(gamma, p):
    """Calculates the relaxation half-life of a quantum system that exchanges energy with its environment.
    This process is modeled via Generalized Amplitude Damping.

    Args:
        gamma (float):
            The probability per unit time of the system losing a quantum of energy
            to the environment.
        p (float): The de-excitation probability due to environmental effect

    Returns:
        (float): The relaxation haf-life of the system, as explained in the problem statement.
    """

    num_wires = 1

    dev = qml.device("default.mixed", wires=num_wires)

    # Put your code here

# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:

    ins = json.loads(test_case_input)
    output = half_life(*ins)

    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, atol=2e-1
    ), "The relaxation half-life is not quite right."

# These are the public test cases
test_cases = [
    ('[0.1,0.92]', '9.05'),
    ('[0.2,0.83]', '7.09')
]
# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[0.1,0.92]'...


JSONDecodeError: Expecting value: line 1 column 1 (char 0)